## Triple-Barrier Method

금융 응용에서 좀 더 현실적인 방법은 포지션의 성패를 나타내는 label을 만드는 것이다. 포트폴리오 매니저가 채택하는 통상적인 거래 규칙은 (1) 미실현 이익목표가 달성되면 그 포지션이 성공으로 마감되며, (2) 미실현 손실 한계에 도달하면 실패로 종결되며, (3) 최대 수의 바까지 그 포지션의 실패도 성공도 없이 청산된다는 것이다. 포지션 성과의 시간 그래프에서 처음 두 조건은 2개의 수평 배리어를 정의하고, 세 번쨰 조건은 수직 배리어를 정의한다. 처음으로 접촉된 배리어와 관련된 바의 인덱스는 $t_{i,1}$로 기록된다. 이익이 나는 배리어를 먼저 건드리면 관측 결과를 $y_i = 1$로 표시한다. loss cut barrier가 먼저 닿으면 관측값을 $y_i = -1$로 표시한다. vertical barrier가 먼저 닿았을 때 우리는 두 가지 옵션을 갖고 있다. 그것을 $y_i = 0$으로 표시하거나 $y_i = \text{sgn}[r_{t_{i, 0}, t_{i,1}}]$으로 표시할 수 있다.